# What drives the price of a car?

![](images/kurt.jpeg)

# Overview

> **Prompt**: In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million
> used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to
> understand what factors make a car more or less expensive.  As a result of your analysis, you should provide
> clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

This was the second Practical Application Project as part of my UC Berkeley ML/AI Professional Certification coursework. 

There were three requirements for this project:

1. Evaluate Linear Regression models for predicting prices for the provided dataset based on the above prompt
2. Use the CRISP-DM Framework as the basis for the project breakdown and deliverables
3. Provide a business-friendly report for our clients at the used car dealership

This Jupyter Notebook follows the CRISP-DM Framework to develop Price Prediction models for our data. 

In addition:

* The initial data investigation is covered in the separate `DataInvestiation.ipynb` notebook and is summarised in the Data section here
  * [Local file](DataInvestigation.ipynb)  |  [Github](https://github.com/fazeelgm/UCB_ML_AI_PracticalApp_II/blob/main/DataInvestigation.ipynb)
* The Executive Summary in the README for this project covers the final report for the clients

~ Fazeel Mufti

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


> **Prompt**: To frame the task, throughout our practical applications we will refer back to a standard process
> in industry for data projects called CRISP-DM.  This process provides a framework for working through a data
> problem.  Your first step in this application will be to read through a brief overview of
> CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After
> reading the overview, answer the questions below.

The main sections of this document have been structured to follow the above CRISP-DM processes to develop the models for predicting auto prices.

### Business Understanding

> **Prompt**: From a business perspective, we are tasked with identifying key drivers for used car prices.  In the
> CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few
> sentences, reframe the task as a data task with the appropriate technical vocabulary.

The dataset provided by the customer contains prices for 426,880 used cars along with 17 attributes for each vehicle. The business objectives for the project are:

1. Provide a price prediction model driven by the feature data
2. Explain which vehicle features drive the price of the car

I will use the `price` column as the **target or dependent variable**, along with the other feature columns, the **independent variables**, as  inputs to a set of LinearRegression models that will use the inputs to predict the price for each vehicle. Each model will be evaluated on the accuracy of its prediction and the best performing model will be used to _interpret the relative importance of each input features to the price_. To objectively measure each model, the data will be divided into a **training** (80%) and a **test set** (remaining 20%). All the models will be trained on the same training set, and will be evaluated against the Test data with known prices. Before training the models, we will remove noise and incomplete information from the dataset so the models can be trained optimally.

# Utilities and Imports

This section imports the necessary dependcies and code needed for our PRice Prediction Models - please expand if needed.

In [6]:
# Customization imports

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

import time

In [7]:
# Export dataFrame's as images
import dataframe_image as dfi

In [8]:
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [9]:
# import my utility methods for this project
import utils_practical_2 as my_utils

In [10]:
# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# logging.getLogger().setLevel(logging.DEBUG)
# logging.getLogger().setLevel(logging.INFO)

In [11]:
# Global format rules

# Seaborn over-rides
sns.set_theme(style='darkgrid')

# The Data

## Data Understanding

> **Prompt**: After considering the business understanding, we want to get familiar with our data.  Write down
> some steps that you would take to get to know the dataset and identify any quality issues within.  Take time
> to get to know the dataset and explore what information it contains and how this could be used to inform your
> business understanding.

**An exploratory data analysis was performed on the Customer dataset and this section summarizes the findings detailed in the companion `DataInvestigation.ipynb` notebook**:
* `DataInvestigation.ipynb`: [Local file](DataInvestigation.ipynb)&nbsp;&nbsp;|&nbsp;&nbsp;[Github](https://github.com/fazeelgm/UCB_ML_AI_PracticalApp_II/blob/main/DataInvestigation.ipynb)

Looking at our `price` target variable, we find:

* There are 32,895 cars with a zero price, 7.71% of the total (426,880) - these are candidates for exclusion pending further analysis
* The price distribution shows an extreme right-skew and a long-tail with a few outliers beyond \\$100K price point
  
![](images/price_distribution.png)

* Analyzed outliers using two separate methods (quantile analysis and Modified z-score)
    * ModZ performed better, i.e. preserved more data samples - removing only 5,790 instead of 8,177 priced vehicles
* Looking at the distribution of priced cars, i.e. cars in the sample dataset that have pricing informaion:
    * There seems to be **at least a low- and mid-price band with a long tail that represents high-priced autos**

Based on Price inspection, a potential hypothesis arose that the used car inventory is _segmented_ based on the following price bands:

![](images/candidate-price-segments.png)

In addition, I did some market research on used car sales for 2023 to learn more about this domain:

> [Auto Remarketing, July 2023: Mid-Year Market Report: Dissecting used-car pricing, segment trends, CPO sales & more](http://digital.autoremarketing.com/publication/frame.php?i=794065&p=&pn=&ver=html5&view=articleBrowser&article_id=4594802)
>
>    * "Average used vehicle transaction price in the first quaretr actually dipped by 6.4% year-over-year to \\$28,381, compared to \\$30,329 in Q1 2022"

Though we don't have year-over-year sales information in this dataset, market research suggests that typical used cars can be categoriezed into price ranges like Budget, Mid, Luxury, etc. that will be based on feature groups. I looked at ```<price, year, condition, odometer>``` combinations and saw that there was clustering behavior as shown by the scatter plots below:

![](images/scatter-price-odo-condition-budget.png)
![](images/scatter-price-odo-year-entry.png)

Please refer to the Data Investigation for more details, but we will use this information to guide our data modeling and optimization.

## Data Preparation

> **Prompt**: After our initial exploration and fine tuning of the business understanding, it is time to
> construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues
> and cleaning, the engineering of new features, any transformations that we believe should happen
> (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

While investigating the target and independent features of our dataset during the Data Investigation, I learnt that the data is very noisy with extreme outliers. Based on my findings, the `get_cleansed_data()` Python function was created, which: 

* Reads the input dataset
* Remove null data and outliers
* Removes unnecessary features like ID, Model and VIN that are specific to individual cars, and are not useful for the kind of machine-learning algorithms that we will be using
* Transforms the datatypes to standard Python types that are more conducive for our regression models

Here is the DataPreparation Report generated by this function:

In [18]:
vehicles_raw, vehicles_cleansed = my_utils.get_cleansed_data(cleanse_data=True)

Reading data/vehicles.csv ... Done: (426880, 18)

Cleansing price column ... 
... Removing price outliers using ModZ method ... 
... ModZ: 9450.0, med: 13950.0, const: 0.6745
... Time: 0.12331819534301758
... Removed 5,790 outliers
... Removing cars with price <= 0 ...  Removed 32,895 rows
Done: (421090, 19) -> (388195, 19)

DropNA from columns: 
... year: 1,029 rows (0.27% of total): 388,195 -> 387,166
... manufacturer: 16,609 rows (4.28% of total): 388,195 -> 371,586
... fuel: 19,173 rows (4.94% of total): 388,195 -> 369,022
... title_status: 26,730 rows (6.89% of total): 388,195 -> 361,465
... odometer: 28,960 rows (7.46% of total): 388,195 -> 359,235
... transmission: 30,742 rows (7.92% of total): 388,195 -> 357,453
Done: (388195, 19) -> (360700, 19)

Dropping columns: ['mod_zscore', 'id', 'model', 'VIN']
... mod_zscore
... id
... model
... VIN
Done: (360700, 19) -> (360700, 15)

Data Transformations:
... year float -> int: Done
... odometer float -> int: Done

Category Transformat

In summary, 66,180 samples were dropped, preserving 84.50% of the original data. It is important to note that we were able to preserve a good portion of high-quality data with `price` information. This gives us a good starting point.

In [20]:
vehicles_cleansed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 360700 entries, 27 to 426879
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   region        360700 non-null  object  
 1   price         360700 non-null  int64   
 2   year          360700 non-null  int64   
 3   manufacturer  360700 non-null  category
 4   condition     223668 non-null  category
 5   cylinders     213522 non-null  category
 6   fuel          360700 non-null  category
 7   odometer      360700 non-null  int64   
 8   title_status  360700 non-null  category
 9   transmission  360700 non-null  category
 10  drive         252582 non-null  category
 11  size          104338 non-null  category
 12  type          282300 non-null  category
 13  paint_color   257706 non-null  category
 14  state         360700 non-null  category
dtypes: category(11), int64(3), object(1)
memory usage: 17.5+ MB


In [21]:
vehicles_cleansed.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
region,360700,404,columbus,3170,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,360700.0,NaN,NaN,NaN,18091.568775,13071.727526,1.0,7000.0,14999.0,26990.0,62981.0
year,360700.0,NaN,NaN,NaN,2011.178007,9.122806,1900.0,2008.0,2013.0,2017.0,2022.0
manufacturer,360700,42,ford,61351,NaN,NaN,NaN,NaN,NaN,NaN,NaN
condition,223668,6,good,113394,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cylinders,213522,8,6 cylinders,81791,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fuel,360700,5,gas,308197,NaN,NaN,NaN,NaN,NaN,NaN,NaN
odometer,360700.0,NaN,NaN,NaN,98543.503208,195802.978631,0.0,39283.0,88632.0,136079.0,10000000.0
title_status,360700,6,clean,348127,NaN,NaN,NaN,NaN,NaN,NaN,NaN
transmission,360700,3,automatic,281732,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This is our candidate dataset that we will use for testing out different Pricing Models!

# Modeling

> **Prompt**: With your (almost?) final dataset in hand, it is now time to build some models.  Here, you
> should build a number of different regression models with the price as the target.  In building your
> models, you should explore different parameters and be sure to cross-validate your findings.

## Feature Preparation for Modeling

I will now evaluate the cleansed data to analyze the features for model training.

But first, let's make a copy of the cleansed data so the _clean data_ is always available for comparison.

In [26]:
# Create a copy of the clean data
data = vehicles_cleansed.copy()

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 360700 entries, 27 to 426879
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   region        360700 non-null  object  
 1   price         360700 non-null  int64   
 2   year          360700 non-null  int64   
 3   manufacturer  360700 non-null  category
 4   condition     223668 non-null  category
 5   cylinders     213522 non-null  category
 6   fuel          360700 non-null  category
 7   odometer      360700 non-null  int64   
 8   title_status  360700 non-null  category
 9   transmission  360700 non-null  category
 10  drive         252582 non-null  category
 11  size          104338 non-null  category
 12  type          282300 non-null  category
 13  paint_color   257706 non-null  category
 14  state         360700 non-null  category
dtypes: category(11), int64(3), object(1)
memory usage: 17.5+ MB


Evaluating features, we see that `region`, `manufacturer`, `paint_color` and `state` have interesting information, but porbably not useful for overall pricing at this stage - we are not developing a regionally-optimized model and want to be able to predict pricing over a large population of used cars. We will drop these features from model training.

In [29]:
# Remove unwanted columns
drop_cols = ['region', 'manufacturer', 'paint_color', 'state']
data.drop(columns=drop_cols, inplace=True)

Let's look at null-data values now ...

In [31]:
print('Percent non-null values in each column')
data.isnull().sum().transform(lambda x: (1 - (x / vehicles_raw.shape[0])) * 100).round(2)

Percent non-null values in each column


price           100.00
year            100.00
condition        67.90
cylinders        65.52
fuel            100.00
odometer        100.00
title_status    100.00
transmission    100.00
drive            74.67
size             39.95
type             81.63
dtype: float64

In [32]:
# DataFrame containing rows with any null values
data_with_nulls = data[data.isnull().any(axis=1)]

# DataFrame containing rows with all non-null values
data_without_nulls = data[~data.isnull().any(axis=1)]

# Print the resulting DataFrames
print('Data with nulls - shape{}: {:,.2f}% of cleansed data'.format(data_with_nulls.shape, (data_with_nulls.shape[0] / vehicles_cleansed.shape[0]) * 100))
print('Data without nulls - shape{}: {:,.2f}% of cleansed data'.format(data_without_nulls.shape, (data_without_nulls.shape[0] / vehicles_cleansed.shape[0]) * 100))

Data with nulls - shape(282074, 11): 78.20% of cleansed data
Data without nulls - shape(78626, 11): 21.80% of cleansed data


Let's remove the nulls from the data to improve model training.

In [34]:
# remove nulls before splitting data

dropna_cols = ['condition', 'cylinders', 'drive', 'type']

print('Dropping nulls: {} -> '.format(data.shape), end='')
# data.dropna(subset=['condition', 'cylinders', 'drive', 'size', 'type'], axis='index', inplace=True)
data.dropna(subset=dropna_cols, axis='index', inplace=True)
print(' {}: {:,.2f}% reduction'.format(data.shape, (1 - (data.shape[0] / vehicles_cleansed.shape[0])) * 100))

Dropping nulls: (360700, 11) ->  (122682, 11): 65.99% reduction


In [35]:
data.columns

Index(['price', 'year', 'condition', 'cylinders', 'fuel', 'odometer',
       'title_status', 'transmission', 'drive', 'size', 'type'],
      dtype='object')

### Create Train/Test splits

We will now create two feature matrices for the independent (X) and target (y) variables., and then split them into Train / Test data sets after encoding all categorical data so the regression models can accept it.

In [38]:
# Create the feature matrices for independnet and target variables

X = data.drop('price', axis='columns')
y = data['price']

In [39]:
# OneHot Encode the features and drop the first value to reduce multicollinearity
X = pd.get_dummies(X, drop_first=True)

In [40]:
# Ensure that we have consistent random-state for all models, otherwise the vary across runs
random_state = 13

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [42]:
# spot-check feature encoding
X.T.iloc[0:10, 0:10]

,31,32,33,34,35,36,38,42,45,55
year,2013,2012,2016,2019,2016,2011,2017,2016,2018,2004
odometer,128000,68696,29499,43000,17302,30237,30041,9704,37332,88000
condition_fair,False,False,False,False,False,False,False,False,False,False
condition_good,False,True,True,False,True,True,True,True,True,True
condition_like new,False,False,False,False,False,False,False,False,False,False
condition_new,False,False,False,False,False,False,False,False,False,False
condition_salvage,False,False,False,False,False,False,False,False,False,False
cylinders_12 cylinders,False,False,False,False,False,False,False,False,False,False
cylinders_3 cylinders,False,False,False,False,False,False,False,False,False,False
cylinders_4 cylinders,False,False,False,False,False,False,False,False,False,False


### Feature Scaling

In [44]:
# Scale the data - we'll use StandardScaler for the baseline model
logging.debug('Scaling data')
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

## Baseline Model

We will use the `DummyRegressor` as the baseline price prediction model, using a simple strategy of always using the `mean` price of the training set as the predicted price.

In [47]:
y_train_mean_price = y_train.mean()
print(f'The mean price for the training dataset is : ${y_train_mean_price:,.2f}')

The mean price for the training dataset is : $15,690.10


Please note that the code below will use a helper method `get_model_metrics_as_results()` that is defined in the `utils_practical_2.py` Python file:

In [49]:
help(my_utils.get_model_metrics_as_results)

Help on function get_model_metrics_as_results in module utils_practical_2:

get_model_metrics_as_results(name, clf, scaler, X_test, y_test, override_rmse=0)
    Build standardized results row given the predictions and y_test values
    
    :param name: Model name for labeling the row in the table
    :param clf: Fitted classifier to get metrics
    :param scaler: Fitted scaler used for transformation
    :param X_test: Test data used to fit the classifier
    :param y_test: Evaluation data to be used for the metrics
    :param override_rmse: Default 0 will internally calculate RMSE as np.sqrt(MSE), pass in a 
        value for cross-validated estimator result if needed
    :return: Returns single row of results summary table containing:
    
        [model_name, MAE, MSE, RMSE, R2_Score, y-intercept]



In [50]:
# Setup the linear models
models = {
    'DummyRegressor' : DummyRegressor(strategy='mean'),
    # 'LinearRegression' : LinearRegression(),
    # 'Ridge' : Ridge(alpha=1.0),
    # 'Lasso' : Lasso(alpha=0.1, max_iter=2000)
}

# save results for tabulation
results_baseline = []

In [51]:
# Run the baseline models

# logging.getLogger().setLevel(logging.DEBUG)

results_baseline = []

# iterate over the models and build results DF
for model in models:

    start_time = time.time()
    print(f'Working on {model} ... ', end='')

    # Fit the model
    models[model].fit(X_train_scaled, y_train)
    
    # Append model metrics to results
    results_baseline.append(my_utils.get_model_metrics_as_results(model, models[model], scaler, X_test_scaled, y_test))
    
    print(f'Done with {model} ({time.time() - start_time:,.4f} sec)')

# logging.getLogger().setLevel(logging.INFO)

Working on DummyRegressor ... Done with DummyRegressor (0.0209 sec)


In [52]:
# Generate results table
results_baseline_df = pd.DataFrame(results_baseline, 
                                   columns=['Model', 'Scaler', 'MAE', 'MSE', 'RMSE', 'R2 Score', 'Base Price']
                                  ).set_index('Model')

# Sort by highest R2, lowest MAE
results_baseline_df.sort_values(by=['R2 Score', 'MAE'], ascending=[False, True], inplace=True)

# Export results for README
results_baseline_df_styled = my_utils.df_style_floats(results_baseline_df)
dfi.export(results_baseline_df_styled, 'images/results_baseline_table.png')

print('RESULTS: Baseline metrics without any optimization (Sorted: max(R2), min(MAE)\n')
results_baseline_df_styled

RESULTS: Baseline metrics without any optimization (Sorted: max(R2), min(MAE)



,Scaler,MAE,MSE,RMSE,R2 Score,Base Price
Model,,,,,,
DummyRegressor,StandardScaler,"10,125.928777","152,738,375.600850","12,358.736813",-0.000027,0


As expected, the R2 score of near 0 shows that the model is no better than a always predicting the mean of the target variable `price`. The slight negative value suggests that our model is slightly worse than simply predicting the mean. There is not much point in interpreting the rest of the metrics at this point.

## Model Tuning: Hyperparameter Tuning and Cross Validation

Now that we have established a baseline, I will use the following standard linear regression models to train our data:

* LinearRegression Model: Provides a baseline for a commercial-strength model, but with no regularization. It should perform well if our dataset is not prone to overfitting
* Lasso (L1 Regularization) Model: Helpful in eliminating less important features, and can be used for feature selection
* Ridge (L2 Regularization) Model: Often performs better than linear regression when multicollinearity is present, as it penalizes large coefficients

In order to find the most suitable model for our data, I will use the following procedure to find the best model:

* Hyperparameter Tuning: GridSearchCV is used to find the best alpha parameter for Ridge and Lasso models. The grid search is performed with 5-fold cross-validation
* Cross-validation: cross_val_score is used to evaluate the models' performance with cross-validation, providing a more robust estimate of model performance
* Model Evaluation: The models are evaluated using Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared metrics on the test set
* Scaling: Use different scaling techniques to bring all the features closer to a similar or same range or scale, so they contribute equally to price prediction. Some models also converge quicker with scaled data
  * StandardScaler: Results in a distribution with 0 mean and 1 standard deviation. Sensitive to outliers
  * MinMaxScaler: Restricts value range to [0,1]. Sensitive to outliers
  * RobustScaler: Removes the median and uses the interquartile range (IQR) as a reference point for scaling. It is not affected by outliers

I will then compare the performance of these models with hyperparameter tuning and cross-validation, providing insights into which model is best suited for predicting car prices.

In [56]:
# Initialize the data structures to loop on for optimization
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge()
}

# hyperparameter settings that we would like to tune for each model
cv_param_grid = {
    'Ridge': {'alpha': [0.1, 1.0, 10.0, 100.0], 'random_state': [random_state]},
    'Lasso': {'alpha': [0.01, 0.1, 1.0, 10.0], 'random_state': [random_state], 'max_iter': [2000] }    
}

# List of scalers we want to try
scalers = {
    StandardScaler(),
    RobustScaler(),    # default: quantile_range=(25, 75)
    MinMaxScaler()
}

In [57]:
# Toggle debugging output
# logging.getLogger().setLevel(logging.DEBUG)
# logging.getLogger().setLevel(logging.INFO)

In [58]:
#
# Set loops to cross-validate over model / scaler combinations using GridSearchCV
#

# save results for tabulation
results_tuned = []
best_models = {}
best_params = {}

# Cross-validation count
cv=5
verbosity=0

# Loop thru each scaler and transform the data
for scaler in scalers:

    print(f'Working on {scaler} ... ')
    logging.debug(f'Working on {scaler}')

    # Scale X_train
    logging.debug(f'Scaling data using {scaler}')
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    # Loop thru each model, fit/predict and store the results
    for model in models:

        start_time = time.time()

        # print(f'\tWorking on {model} ... ', end='')

        clf = models[model]

        # Handle CV models
        if (model in cv_param_grid):
            print('\tWorking on {}: {}, Scaler: {}'.format(model, cv_param_grid[model], scaler))
            grid = GridSearchCV(clf, param_grid=cv_param_grid[model], 
                                cv=cv, scoring='neg_mean_squared_error', verbose=verbosity)
            grid.fit(X_train_scaled, y_train)
              
            # Save the tuning metrics for this model class
            best_models[model] = grid.best_estimator_
            best_params[model] = grid.best_params_
            logging.debug(f'Best param: {grid.best_params_}')            
            rmse = np.sqrt(-grid.best_score_)

        # Handle non-CV models
        else:
            print('\tWorking on {}, Scaler: {}'.format(model, scaler))
            clf.fit(X_train_scaled, y_train)
            cv_scores = cross_val_score(clf, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

            # Save the tuning metrics for this model class
            best_models[model] = clf
            best_params[model] = ''
            rmse = np.sqrt(-np.mean(cv_scores))

        y_preds = best_models[model].predict(X_test_scaled)

        # Get the predictions for this model instance
        score = best_models[model].score(X_test_scaled, y_test)
        r2 = r2_score(y_test, y_preds)
        logging.debug(f'CV Best RMSE: {rmse}')
        logging.debug(f'Score: {score}, r2: {r2}')
       
        # Append model metrics to results
        results_tuned.append(my_utils.get_model_metrics_as_results(model, best_models[model], scaler, X_test_scaled, y_test, override_rmse=rmse))

        print(f'\tDone with {model} ({time.time() - start_time:,.4f} sec)')

    print(f'Done with {scaler}')

logging.getLogger().setLevel(logging.INFO)

Working on MinMaxScaler() ... 
	Working on LinearRegression, Scaler: MinMaxScaler()
	Done with LinearRegression (4.5714 sec)
	Working on Lasso: {'alpha': [0.01, 0.1, 1.0, 10.0], 'random_state': [13], 'max_iter': [2000]}, Scaler: MinMaxScaler()
	Done with Lasso (30.8186 sec)
	Working on Ridge: {'alpha': [0.1, 1.0, 10.0, 100.0], 'random_state': [13]}, Scaler: MinMaxScaler()
	Done with Ridge (0.8664 sec)
Done with MinMaxScaler()
Working on StandardScaler() ... 
	Working on LinearRegression, Scaler: StandardScaler()
	Done with LinearRegression (0.7658 sec)
	Working on Lasso: {'alpha': [0.01, 0.1, 1.0, 10.0], 'random_state': [13], 'max_iter': [2000]}, Scaler: StandardScaler()
	Done with Lasso (23.1111 sec)
	Working on Ridge: {'alpha': [0.1, 1.0, 10.0, 100.0], 'random_state': [13]}, Scaler: StandardScaler()
	Done with Ridge (0.8808 sec)
Done with StandardScaler()
Working on RobustScaler() ... 
	Working on LinearRegression, Scaler: RobustScaler()
	Done with LinearRegression (0.7498 sec)
	Work

In [ ]:
# Generate results table
results_tuned_df = pd.DataFrame(results_tuned,
                                columns=['Model - Tuned', 'Scaler', 'MAE', 'MSE', 'RMSE', 'R2 Score', 'Base Price']
                               ).set_index('Model - Tuned')

# Add BestParams as a new column
results_tuned_df['CV Best Params'] = results_tuned_df.index.map(best_params)

# Sort by highest R2, lowest MAE
results_tuned_df.sort_values(by=['R2 Score', 'MAE'], ascending=[False, True], inplace=True)

# Export results for README
results_tuned_df_styled = my_utils.df_style_floats(results_tuned_df)
dfi.export(results_tuned_df_styled, 'images/results_tuned_table.png')

# Evaluation

> **Prompt**: With some modeling accomplished, we aim to reflect on what we identify as a high quality model
> and what we are able to learn from this.  We should review our business objective and explore how well we
> can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings
> and determine whether the earlier phases need revisitation and adjustment or if you have information of
> value to bring back to your client.

## Picking the Best Model

Now that we have the results for our baseline and candidate models in a standardized format, let's evaluate them on performance:

In [ ]:
print('RESULTS: Baseline Model metrics without any optimization (Sorted: max(R2), min(MAE)\n')
results_baseline_df_styled

In [ ]:
print(f'RESULTS: Optimized Model results after hyperparameter tuning and cross-validation ({cv}-fold): Sorted max(R2), min(MAE)\n')
results_tuned_df_styled

In [ ]:
best_model_idx = 0   # ASSUMES SORTED LIST
results_row = results_tuned_df.iloc[best_model_idx]
best_model_name = results_tuned_df.index[best_model_idx]
best_model = best_models[best_model_name]
y_intercept = np.float64(results_row['Base Price'].replace(',',''))

print('Best Model with highest R2 Score: {}\n\tBest Params: {}\n\tScaler: {}\n\tR2 Score: {:,.2f}%\n\tRMSE: ${:,.2f}\n\tBase Price (y-intercept): ${:,.2f}'
      .format(best_model_name,
              results_row['CV Best Params'],
              results_row['Scaler'],
              float(results_row['R2 Score']) * 100, 
              float(results_row['RMSE'].replace(',','')),
              y_intercept))

We see that all the models performed very closely without any clear winners, with **the best R2 score of 52.07% from the tuned Ridge regression model**. Compared to the `DummyRegressor` as our baseline where (R2 = 0) it performed the same as simply predicting the mean of all prices, our tuned Ridge model explains 52% of the variability when predicting the car price from the features, with a standard deviation of $8,496.24 (RMSE: root mean squared error) from the actual price. This is a good start!

We can also make the following observations:

* The base price of a car is given by the y-intercept for the linear regression model: $15,690.10
* Each training feature contributes to the predicted price based on the weights for that specific feature that the model has learned from the training data - we will investigate this further later

**_We will use the best performing Ridge model as the basis of our evaluation from this point forward_**.

### Visualizing Actual vs Predicted Prices

Let's visualize our predicted prices relative to the actual prices in the training set.

In [ ]:
# Plot predictions
y_preds = best_model.predict(X_test_scaled)

# Adjust for the y-intercept
y_preds = np.add(y_preds, y_intercept)
perfect_pt_1 = [min(y_test), max(y_test)]
perfect_pt_2 = [min(y_test), max(y_test)] + y_intercept

plt.figure(figsize=(12, 6))
plt.scatter(x=y_test, y=y_preds, label=f'{best_model_name} Model', alpha=0.5)
plt.plot(perfect_pt_1, perfect_pt_2, color='red', linestyle='--', label='Theoretical Perfect Predictions')

plt.xlabel('Actual Prices', fontdict=my_utils.axes_fonts)
plt.ylabel('Predicted Prices', fontdict=my_utils.axes_fonts)
plt.title(f'Model Predictions vs Actual Prices for the best performing {best_model_name} Model (y-intercept=${y_intercept:,.2f})', 
          fontdict=my_utils.title_fonts)
plt.legend()
plt.tight_layout()

plt.savefig('images/scatter-optimized-preds-v-test.png')
plt.show()

The red line represents the ideal case if all our predictions perfectly matched the actual prices. From the scatter plot above, we can see that our model has learnt the basic pricing relationship from our features but is not performing well on the top and bottom of the price range. 

### PredictionErrorDisplay: Residuals Comparison

Let's look at our prediction data in a different manner to understand how much of the price variability is being captured by our features:

In [ ]:
# # How to get the list of available Displays for my version of sklearn
# from sklearn.utils.discovery import all_displays
# displays = all_displays()
# displays

from sklearn.metrics import PredictionErrorDisplay

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
PredictionErrorDisplay.from_estimator(best_model, X_test_scaled, y_test, ax=axes[0], kind="actual_vs_predicted")
PredictionErrorDisplay.from_estimator(best_model, X_test_scaled, y_test, ax=axes[1], kind="residual_vs_predicted")

plt.suptitle(f'Best Model: Comparing Predictions vs Actual Prices (without y-intercept=${y_intercept:,.2f})')
plt.tight_layout()

plt.savefig('images/optimized-PredictionErrorDisplay.png')
plt.show()

The graph on the left is showing us that the for the high-end, the prediction error increases from the perfect center line. Ideally, the scatter points would hug the diagonal. The graph on the right shows us the residuals, the difference between the actual and the predicted value, and shows us that the model is performing better in the middle price ranges. 

Taken together, this shows us that we should perhaps use non-linear regresssion methods for this dataset to capture the variation on each end of the price spectrum. However, for the purpose of this study, we're deliberately restricting ourselves to linear regression models to learn more about them.

## Interpreting the Best Model Results

#### Feature Importance

We now switch our focus to interpreting the results of the best performing model and analyze the coefficients, or weightings, for each feature that were _learnt_ by our model during the training process. This represents the causal relationship between that feature and the price of the vehicle, or the importance of that feature to the price prediction.

In [ ]:
# logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
# build the feature importance DF

feature_names = X.columns

feature_imp = []
for feature, coef in zip(feature_names, best_model.coef_):
    if coef != 0:
        # print(f'{feature}: {coef}')
        feature_imp.append([feature, coef])

feature_imp_df = pd.DataFrame(data=feature_imp, columns=['Feature', 'Coefficient'])#.set_index('Feature')
feature_imp_df['Importance'] = np.abs(feature_imp_df.Coefficient)

# Sort the DF
feature_imp_df = feature_imp_df.sort_values(by='Importance', ascending=False)

# Add cumulative importance
feature_imp_df['Cumulative Importance'] = feature_imp_df['Importance'].cumsum() / feature_imp_df['Importance'].sum()
feature_imp_df['Percent Contribution'] = feature_imp_df['Cumulative Importance'] * 100

top_n = 10
print(f'Impact of top-{top_n} features to the Base Price: ${y_intercept:,.2f} (y-intercept)')
feature_imp_df_styled = my_utils.df_style_floats(feature_imp_df[['Feature', 'Coefficient', 'Percent Contribution']]
                                                 .set_index('Feature').rename(columns={'Coefficient':'Price Impact'}))

# logging.getLogger().setLevel(logging.INFO)
feature_imp_df_styled.head(top_n)

In [ ]:
# Top features that explain X% of the variance
top_var = .85
top_features = feature_imp_df[feature_imp_df['Cumulative Importance'] <= top_var]

# Plot feature importance
plt.figure(figsize=(8, 8))
plt.barh(top_features['Feature'], top_features['Coefficient'])
plt.axvline(x=0, color="k", linestyle="-", label=f'Represents Base Price: ${y_intercept:,.2f}')

plt.xlabel(f'Positive or Negative Impact of Individual Feature on Base Price', fontdict=my_utils.axes_fonts)
plt.ylabel('Feature by Importance on Price', fontdict=my_utils.axes_fonts)

plt.title(f'Top Features Explaining {top_var * 100:,.0f}% of Price Variance', fontdict=my_utils.title_fonts)
plt.gca().invert_yaxis()
plt.legend(loc='lower left')
plt.tight_layout()

plt.savefig('images/feature_importance.png')
plt.show()

From the feature importance provided by the trained model, we can see the top features that capture upto 85% of the price variation, and individually represent the impact to price when there is a one-unit change in the corresponding feature. For example, our model is aware of the different `condition` values when predicting prices, with `condition_fair` negatively impacting price (-6,695.25) and `condition_new` positively impacting price.

#### Permutation Importance

Now that we have seen which features impact the price the most, let's calculate the permutation importance to measure the change in our model's performace when a feature value is randomly shuffled to see how much the model relies on that feature for its predictions. This will help us determine collinearity between features not captured by the model cofficients and evaluate the impact of changing the feature on the model performance.

In [ ]:
from sklearn.inspection import permutation_importance

p_imp = permutation_importance(best_model, X_test_scaled, y_test, n_repeats=10, random_state=random_state)
p_imp_mean = p_imp.importances_mean
p_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': p_imp_mean})
p_imp_df.sort_values(by='Importance', ascending=False).head()

In [ ]:
def plot_permutation_importance(p_imp_result, features, p_imp_mean, ax):

    perm_sorted_idx = p_imp_mean.argsort()

    ax.boxplot(
        p_imp_result.importances[perm_sorted_idx].T,
        vert=False,
        tick_labels=X.columns[perm_sorted_idx],
    )
    ax.axvline(x=0, color="k", linestyle="--")
    return ax

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot_permutation_importance(p_imp, feature_names, p_imp_mean, ax)
ax.set_title("Permutation Importances on multicollinear features", fontdict=my_utils.title_fonts)
ax.set_xlabel("Decrease in accuracy score")
_ = ax.figure.tight_layout()

plt.savefig('images/permutation_importance.png')
plt.show()

#### Comparing Feature / Permutation Importance

We see that there are slight differences between the Feature and Permuation Importance, so let's plot them side-by-side to better explain feature elimination to our customer.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 10))

fig.suptitle('Feature vs. Permutation Importance', fontdict=my_utils.title_fonts)
# feature importance
ax1.barh(feature_imp_df['Feature'], feature_imp_df['Coefficient'])
ax1.axvline(x=0, color="k", linestyle="-", label=f'Base Price: ${y_intercept:,.2f}')

# ax1.set_xlabel(f'Pos / Neg Impact of Individual Feature on Base Price', fontdict=my_utils.axes_fonts)
# ax1.set_ylabel('Feature by Importance on Price', fontdict=my_utils.axes_fonts)

ax1.set_title('')
ax1.invert_yaxis()
ax1.legend(loc='lower left')

# perm imp
plot_permutation_importance(p_imp, feature_names, p_imp_mean, ax2)
ax2.set_title('')
# ax2.set_xlabel("Decrease in accuracy score", fontdict=my_utils.axes_fonts)

_ = ax.figure.tight_layout()

plt.savefig('images/feature_perm_importance.png')
plt.show()

### Utility Functions: Segment Plotting

Before analyzing the Feature and Permutation Importance information, let's define a couple of utility functions that facilitate plotting and tabulating the results in a standardized manner:

* `fit_plot_segments`: Generate a scatter plot of Actual vs Predicted Prices given a standardized data structure containing criteria-based data segments. For each data segment:
  * Create and scale train/test sets,
  * Fit the training set and predict using the test set
  * Returns a data structure to tabulate the metrics 
* `generate_segments_table`: Tabulates the results for each data segment from the above function

These functions will be used for different segment analysis from this point forward and can be found in the `utils_practical_2.py` Python module.

In [ ]:
help(my_utils.fit_plot_segments)

In [ ]:
help(my_utils.generate_segments_table)

Together, the graphs above show us the complete picture of the important features that determine price predictions.

## Feature Elimination & Optimization

Based on the above side-by-side comparison of the feature and permutation importance, we can see that a majority of the price is determeined by a relatively few features. Taking a look at the features that drive 85% of the price variance, the following features stand out, with different values impacting the price:

* Odometer
* Year
* Condition
* Transmission
* Type
* Drive
* Fuel
* Cylinders

Based on the permutation importance, `year` pops to the top of the list. These results are intuitive and make sense from what we consider to be important considerations when pricing a used car.

The other features can be eliminated as they are mostly colinear and detract from the accuracy score.

Interestingly:

* `title_status` was eliminated likely due to the vast majority of the cars having a 'clean' title
* `size` is likely being over-shadowed by `transmission` and `cylinders` due to colinearity

Let's run the best model again after removing the unnecessary features: `title_status`, `size`.

In [ ]:
drop_cols = ['title_status', 'size']

# drop_cols = ['title_status', 'cylinders','transmission']
# drop_cols = ['title_status']
data.drop(columns=drop_cols, inplace=True)

In [ ]:
data.info()

Now we have narrowed down our focus to the above features. We recall from our initial data investigation that both `year` and `odometer` were identified as components of typical market segmentation of used cards. Let's analyze the distribution of these two features to check for outliers:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 0.5))
fig.suptitle('Odometer / Year highly skewed distribution before cutoffs', y=1.3)
sns.boxenplot(ax=ax1, data=data, x='odometer')
ax1.set_title('')

sns.boxenplot(ax=ax2, data=data, x='year')
ax2.set_title('')

_ = ax.figure.tight_layout()
plt.show()

We see unrealistically skewed data that will affect our model training, so I will look for logical cutoffs to preserve data but also make it realistic from a pricing perspective:

In [ ]:
# Define the odometer cutoff ranges for lo/hi milage
odo_cutoff_lo = 0
lo = data.query('odometer <= @odo_cutoff_lo')
odo_cutoff_hi = 250000

# Define year cutoff
year_cutoff = 1980
year_cutoff_lo = data.query('year <= @year_cutoff')
temp = data.query('odometer > @odo_cutoff_lo and odometer <= @odo_cutoff_hi and year >= @year_cutoff')
print('Odometer Cutoff Distribution: {:,d} ({:,.2f}%) <= lo cutoff({:,d} miles) <= {:,d} ({:,.2f}%) < hi cutoff ({:,d} miles) < {:,d} ({:,.2f}%)'
      .format(lo.shape[0], ((lo.shape[0] / data.shape[0]) * 100), 
              odo_cutoff_lo,
              temp.shape[0], ((temp.shape[0] / data.shape[0]) * 100),
              odo_cutoff_hi, 
              data.shape[0] - temp.shape[0], ((1 - temp.shape[0] / data.shape[0]) * 100)))

In [ ]:
print('Year Cutoff Distribution: {:,d} ({:,.2f}%) <= cutoff({:d}) <= {:,d} ({:,.2f}%)'
      .format(year_cutoff_lo.shape[0], ((year_cutoff_lo.shape[0] / data.shape[0]) * 100), 
              year_cutoff,
              temp.shape[0], ((temp.shape[0] / data.shape[0]) * 100)))

In [ ]:
data=temp

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 0.5))
fig.suptitle('Odometer / Year distribution after cutoffs', y=1.3)
sns.boxenplot(ax=ax1, data=data, x='odometer')
ax1.set_title('')

sns.boxenplot(ax=ax2, data=data, x='year')
ax2.set_title('')

_ = ax.figure.tight_layout()
plt.show()

I decided to set the following cutoffs to get a more realistic inventory for re-trainig our best model:

* Odometer: Restricted to milage between 0 and 250,000 miles
  * Removed 232 low milage cars (0.19% of total) and 5,554 high-milage cars (4.53% of total)
* Year: Removed 2,973 cars (2.42% of total) older than the 1980 model year

Now, we're ready to re-train the model and analyze the results.

NOTE: We will be using the segment reporting utility functions mentioned above: `fit_plot_segments` and `generate_segments_table`

In [ ]:
# Data setup: Eliminated Features segment

elim_segments = {
    'Optimized Features': data
}

graph_x_label = 'Actual Prices'
graph_y_label = 'Predicted Prices'
elim_title = 'Optimized Features: Predictions vs Actuals'

segments_dict = {
    'Optimized Features': {
        'seg_data': elim_segments,
        'graph_title': elim_title,
        'graph_x_label': graph_x_label,
        'graph_y_label': graph_y_label,
        'graph_filename': 'images/scatter-segments-opt-preds-v-test.png',
        'seg_results': [],
        'results_table_title': elim_title,
        'results_table_filename': 'images/results_opt_segments.png'
    }
}

In [ ]:
# Plot segments in single graph
segment = 'Optimized Features'
segments_dict[segment]['seg_results'] = my_utils.fit_plot_segments(segments_dict[segment], 
                                                                   segments_dict[segment]['seg_results'],
                                                                   best_model, scaler, random_state,
                                                                   segments_dict[segment]['graph_filename'])

In [ ]:
# generate results table
seg_df = my_utils.generate_segments_table(segment, 
                                          segments_dict[segment]['seg_results'], 
                                          png_filename=segments_dict[segment]['results_table_filename'])

print(segments_dict[segment]['results_table_title'])
seg_df

We can see from the graph and the results table that our best Ridge Model really improved dramatically after optimization from 52.07% R2 score to 71.07%. In addition, the Average Price across all predictions is a more realistic \\$30,531.

## Model Applications: Segment Analysis

Now that the model has been re-trained, we can validate it using some of the findings from our data investigation where we found different ways to segment the pricing data. We will use our optimized model to predict prices for cars in these segments and see how our model performs. Please note that these new test segments includes cars from both the training and test sets, and we want to use this exercise as examples of interesting application of our Pricing Model.

### Market Defined Segments: Pricing Guidance

We will now run the Pricing Model against a test set defined on the following market-based classification of our cars:

* Budget: Model Year <= 2016 and Odometer >= 80,000
* Entry: Model Year between 2016 and 2019, Odometer between 60,000 and 80,000
* Mid: Model Year between 2020 and 2021, Odometer between 30,000 and 60,000
* Premium: Model Year newer than 2021, Odometer less than  30,000

This is typical of how used cars are categorized on the dealer lot. Let's see how we do! 

In [ ]:
# Market Defintion of car segementations typical in the used car sales business

mkt_segments = {
    'Budget (YR <= 2016, Odo >= 80,000)': data.query('year <= 2016 and odometer >= 80000'),
    'Entry (2016 >= YR <= 2019, 60,000 >= ODO <= 80,000)': data.query('(year>=2016 and year<=2019) and (odometer>=60000 and odometer<=80000)'),
    'Mid (2019 >= YR <= 2021, 30,000 >= ODO <= 60,000)': data.query('(year>=2019 and year<=2021) and (odometer>=30000 and odometer<=60000)'),
    'Premium (YR >= 2021, ODO >= 30,000)': data.query('(year>=2021 and year<=2024) and (odometer<=30000)'),
}

graph_x_label = 'Actual Prices'
graph_y_label = 'Predicted Prices'
mkt_title = 'Market-defined Segments: Predictions vs Actuals'

segments_dict = {
    'Market-defined Segments': {
        'random_state': random_state,
        'seg_data': mkt_segments,
        'graph_title': mkt_title,
        'graph_x_label': graph_x_label,
        'graph_y_label': graph_y_label,
        'graph_filename': 'images/scatter-segments-mkt-preds-v-test.png',
        'seg_results': [],
        'results_table_title': mkt_title,
        'results_table_filename': 'images/results_mkt_segments.png'
    }
}

In [ ]:
# Plot segments in single graph
segment = 'Market-defined Segments'
segments_dict[segment]['seg_results'] = my_utils.fit_plot_segments(segments_dict[segment], 
                                                                   segments_dict[segment]['seg_results'],
                                                                   best_model, scaler, random_state,
                                                                   segments_dict[segment]['graph_filename'])

In [ ]:
# generate results table
seg_df = my_utils.generate_segments_table(segment, 
                                          segments_dict[segment]['seg_results'], 
                                          png_filename=segments_dict[segment]['results_table_filename'])

print(segments_dict[segment]['results_table_title'])
seg_df

As we can see, our model scores fairly well for the first three segments and is able to predict reasonable average prices for each segment. This is an example of using the Pricing Model in **Pricing Guidance** use cases for new inventory, where sales agents can input the vehicle features into the model to get suggested prices.

### Price-based Segments: Sale Price Validation

Next, we define a test set based on the following price-based classification of the cars:

* Low-priced cars: Price < 5,000
* Mid-priced cars: Prices between 5,000 and 50,000
* High-priced cars: Price above 50,000

In [ ]:
# Define segments
mid_boundary = 5000
high_boundary = 50000
price_segments = {
    'Low-priced (Price < 5,000)': data.query('price > 0 and price < @mid_boundary'),
    'Mid-priced (5,000 >= Price < 50,000)': data.query('price >= @mid_boundary and price < @high_boundary'),
    'High-priced (Price >= 50,000)': data.query('price >= @high_boundary'),
}

price_title = 'Price-based Segments: Predictions vs Actuals'

segments_dict = {
    'Price-based Segments': {
        'random_state': random_state,
        'seg_data': price_segments,
        'graph_title': price_title,
        'graph_x_label': graph_x_label,
        'graph_y_label': graph_y_label,
        'graph_filename': 'images/scatter-segments-price-preds-v-test.png',
        'seg_results': [],
        'results_table_title': price_title,
        'results_table_filename': 'images/results_price_segments.png'
    }
}

In [ ]:
# Plot segments in single graph
segment = 'Price-based Segments'
segments_dict[segment]['seg_results'] = my_utils.fit_plot_segments(segments_dict[segment], 
                                                                   segments_dict[segment]['seg_results'],
                                                                   best_model, scaler, random_state,
                                                                   segments_dict[segment]['graph_filename'])

In [ ]:
# generate results table
seg_df = my_utils.generate_segments_table(segment, 
                                          segments_dict[segment]['seg_results'], 
                                          png_filename=segments_dict[segment]['results_table_filename'])

print(segments_dict[segment]['results_table_title'])
seg_df

We see that our model performs even better (73.98%) for the mid-priced range than what we have seen so far. For low- and hi-priced cars, there is likely insufficient data to model these cases. This gives us confidence that we can use this model to build a model to classify newly acquired inventory into the market-based segments from the previous Pricing Guidance example.

### Validation Dataset: Defining the Null Data Segment

If you go back and see how we cleaned the initial data set, we used `dropna()` to remove all nulls from the following columns before modeling to get a clean, high-quality dataset:

In [ ]:
dropna_cols

At that time we had saved a copy of the rows that had atleast one column with a null value but included prices - we can treat this as a *validation dataset* to use our best model to predict prices. This is a good benchmark to see how it would do forecasting pricing for new autos that didn't have all the car attributes defined!

In [ ]:
# Define the data segement for rows with prices, but atleast one null-value column
nulls_segment = {
    'Data with Nulls': data_with_nulls
}

Null_title = 'Null Data Segment: Predictions vs Actuals'

segments_dict = {
    'Null Data Segments': {
        'seg_data': nulls_segment,
        'graph_title': Null_title,
        'graph_x_label': graph_x_label,
        'graph_y_label': graph_y_label,
        'graph_filename': 'images/scatter-segments-nulls-preds-v-test.png',
        'seg_results': [],
        'results_table_title': Null_title,
        'results_table_filename': 'images/results_nulls_segments.png'
    }
}

In [ ]:
# Plot segments in single graph
segment = 'Null Data Segments'
segments_dict[segment]['seg_results'] = my_utils.fit_plot_segments(segments_dict[segment], 
                                                                   segments_dict[segment]['seg_results'],
                                                                   best_model, scaler, random_state,
                                                                   segments_dict[segment]['graph_filename'])

In [ ]:
# generate results table
seg_df = my_utils.generate_segments_table(segment, 
                                          segments_dict[segment]['seg_results'], 
                                          png_filename=segments_dict[segment]['results_table_filename'])

print(segments_dict[segment]['results_table_title'])
seg_df

# Deployment

> **Prompt**: Now that we've settled on our models and findings, it is time to deliver the information to the client.
> You should organize your work as a basic report that details your primary findings.  Keep in mind that your
> audience is a group of used car dealers interested in fine tuning their inventory.


Since we are not deploying this model in a production environment yet, we will provide a Plain Language Report to the customer that summarizes our results using the CRISP-DM format.